# 以下函数用于 [colourlovers](http://www.colourlovers.com/palettes/new/all-time/meta?page=3)

### 启动浏览器引擎 

In [ ]:
import json
from selenium import webdriver
import multiprocessing as mp
browser = webdriver.Chrome('./chromedriver')

### 开始批量下载每个页面上的配色方案

In [ ]:
full_dicts = {}
for m in range(0,2):
    browser.get("https://www.colourlovers.com/palettes/most-loved/all-time/meta?page={}".format(str(m)))
    a=browser.find_elements_by_class_name("detail-row")

    for n,j in enumerate(a):
#         print('**********************************')
        full_dicts[str(m*(n+1))] = []
        b = j.find_elements_by_class_name("c")
        if len(b) == 10:
            for k,i in enumerate(b):
                if k < 5:
                    content = i.get_attribute('style')
                    color = content.split('rgb')[-1].split(';')[0]
#                     print(color)
                    full_dicts[str(m*(n+1))].append(color)
    print('Process Done the %dth page' % m)

### 以json的格式保存数据

In [ ]:
with open('colorlove.json','w',encoding='utf-8') as f:
    json.dump(full_dicts,f,ensure_ascii=False)

# 以下函数用于 [coolors](https://coolors.co/browser/best/1)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome(executable_path="./chromedriver")
# browser = webdriver.Chrome('./chromedriver')
# from pyvirtualdisplay import Display
# display = Display(visible=0,size=(800,600))

In [ ]:
def str2tuple(a):
#     print(a)
    b = a.replace('(','').replace(')','').replace(' ','').split(',')
    return tuple([int(i) for i in b])

def rgb2hex(value):
    digit = list(map(str, range(10))) + list("ABCDEF")
    string = '#'
    for i in value:
        a1 = i // 16
        a2 = i % 16
        string += digit[a1] + digit[a2]
    return string

In [ ]:
full_dicts = {}
count = 0
for i in range(1,2):
    browser.get('https://coolors.co/browser/best/{}'.format(str(i)))
    tag = True


    new_dict = {}
#     palette_names = WebDriverWait(browser, 30).until(
#         EC.presence_of_element_located((By.CLASS_NAME, "browser-palette-name"))
#     )
    
#     palette_colors = WebDriverWait(browser, 30).until(
#         EC.presence_of_element_located((By.CLASS_NAME, "browser-palette-colors"))
#     )
    
    palette_names = browser.find_elements_by_class_name(name='browser-palette-name')
    palette_colors = browser.find_elements_by_class_name(name='browser-palette-colors')
    for browser_name,browser_colors in zip(palette_names,palette_colors):
        name = browser_name.find_element_by_class_name(name='name').text + str(count)
        browser_color = browser_colors.find_elements_by_tag_name('div')
        colors = []
        for color in browser_color:
            color = color.get_attribute('style') # 'background: rgb(36, 123, 160);'
            color = color.split('rgb')[-1].split(';')[0]
            color = rgb2hex(str2tuple(color))
            colors.append(color)
        new_dict[name] = colors
        count += 1
    full_dicts = dict(full_dicts,**new_dict)
    del new_dict
    print('process done page {}...'.format(str(i)))
    tag = False



# 以下函数用于 [配色网](http://peise.net)

In [ ]:
from __future__ import unicode_literals
from urllib import request
import chardet
from bs4 import BeautifulSoup
import re
import json
import os

In [ ]:
import time
a = None
def parsing(dct,url):
    begin = time.time()
    global a
    response = request.urlopen(url)
    html = response.read()
    charset = chardet.detect(html)
    html = html.decode('gbk')
    soup = BeautifulSoup(html, "html.parser")
    parse = soup.find_all('li','listpalette')
#     pages = soup.find_all(href=re.compile("^palette/"))
    pages = soup.find_all(href=re.compile("^http://peise.net/tag.php"))

    for p in parse:
        author_named = p.h3.text
#         print(author_named)
        title = p.h6.text
        
        dct[author_named] = []
        p = p.find_all('span')
        for pp in p:
            pp = pp['style'].split(':')[-1]
            dct[author_named].append(pp)
    # print(dct)
    end = time.time()
    print("It takes %fs to read:%s" % ((end-begin,url)))
    if pages[-1]['href'] == url:
        a = dct
        return
    else:
#         new_url = "http://www.peise.net/"+pages[-2]['href']
        new_url = pages[-2]['href']
#         print(new_url)
        if new_url == url:
            return 
        #new_url = pages[-2]['href']
        parsing(dct,new_url)


In [ ]:
count = {}
with open('use_title.txt','r') as f:
    f = f.readlines()
    root = 'http://peise.net/tag.php?tag='
    for info in f:
        info = info.split()
        path = info[0]
        name = info[1].split('\n')[0]
        dct = {}
        parsing(dct,root+path)
        with open('titles/{}.json'.format(name),'w',encoding='utf-8') as f:
            json.dump(dct,f,ensure_ascii=False)
#         count[name] = len(dct)
        

In [ ]:
with open('titles/{}.json'.format('count'),'w',encoding='utf-8') as f:
    json.dump(count,f,ensure_ascii=False)